In [34]:
import pandas as pd
from glob import glob
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch
import torch.nn as nn
import torch.optim as optim
import os
import math
import shutil
import torch.nn.functional as F

In [3]:
data_path = os.path.join(os.getcwd(), 'csv_final')
data_path_list = glob(os.path.join(data_path, '*.csv'))
print(len(data_path_list))

281


In [4]:
using_columns = ['Time[s]', 'TE1', 'E.RPM',
       'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar','E.Id','RPM', 'P_UD']

x_columns = [ 'TE1', 'E.RPM', 'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar']
#x_columns = [ 'E.Iq']
y_columns = ['P_UD']

In [5]:
data_size = len(data_path_list)
train_size = int(data_size*0.8)
test_size = data_size - train_size
val_size = int(train_size*0.2)

In [6]:
train_datapath, test_datapath = random_split(data_path_list, [train_size, test_size])
train_datapath, val_datapath = random_split(train_datapath, [(train_size - val_size),val_size])

In [7]:
def createFolder(directory): #현재 경로에 Train / Test / Val 폴더 생성
    if not os.path.exists(directory):
        os.mkdir(directory)
        
    elif os.path.exists(directory):
        print('Already exists')
        
    else:
        print('Error: Creating directory. ' + directory)
        
createFolder(os.path.join(os.getcwd(),'csv_final/train'))
createFolder(os.path.join(os.getcwd(),'csv_final/test'))
createFolder(os.path.join(os.getcwd(),'csv_final/val'))

Already exists
Already exists
Already exists


In [8]:
for i in range(len(train_datapath)):
    shutil.copy2(train_datapath[i], './csv_final/train')
    
for j in range(len(test_datapath)):
    shutil.copy2(test_datapath[j], './csv_final/test')
    
for k in range(len(val_datapath)):
    shutil.copy2(val_datapath[k], './csv_final/val')

In [9]:
train=[]
test=[]
val=[]
num = 0

for path in train_datapath :
    num = num + 1
    df_cut = pd.read_csv(path)[using_columns]
    train.append(df_cut)
    if num > 3 :
        break
    
for path in test_datapath :
    df_cut = pd.read_csv(path)[using_columns]
    test.append(df_cut)
    
for path in val_datapath :
    df_cut = pd.read_csv(path)[using_columns]
    val.append(df_cut)

In [10]:
print(train[1])

      Time[s]     TE1  E.RPM  E.Vlt  E.Iq  E.Tmp  E.Pwr   E.Tar  E.Id  RPM  \
0       0.000  24.589  688.0   13.0  2.82   37.0    3.0  2786.0   0.0  0.0   
1       0.001  24.589  688.0   13.0  2.82   37.0    3.0  2786.0   0.0  0.0   
2       0.002  24.589  688.0   13.0  2.82   37.0    3.0  2786.0   0.0  0.0   
3       0.003  24.589  688.0   13.0  2.82   37.0    3.0  2786.0   0.0  0.0   
4       0.004  24.589  688.0   13.0  2.82   37.0    3.0  2786.0   0.0  0.0   
...       ...     ...    ...    ...   ...    ...    ...     ...   ...  ...   
9115    9.115  24.528  688.0   13.0  2.68   37.0    3.0  2798.0   0.0  0.0   
9116    9.116  24.528  688.0   13.0  2.68   37.0    3.0  2798.0   0.0  0.0   
9117    9.117  24.528  688.0   13.0  2.68   37.0    3.0  2798.0   0.0  0.0   
9118    9.118  24.528  688.0   13.0  2.68   37.0    3.0  2798.0   0.0  0.0   
9119    9.119  24.528  688.0   13.0  2.68   37.0    3.0  2798.0   0.0  0.0   

       P_UD  
0     1.541  
1     1.540  
2     1.540  
3     1

In [11]:
def window_torch(df, x_size, shift): # size = 윈도우 사이즈 / shift = 윈도우 시작점 얼만큼 이동할건지
    df_X =False
    for i in range(len(df)):
        if df_X is False:
            df_X = torch.tensor(df[i][x_columns].values, dtype=torch.float32)
            df_X = df_X.unfold(0, x_size, shift).transpose(1,2)
            
        else:
            tmp1 = torch.tensor(df[i][x_columns].values, dtype=torch.float32)
            tmp1 = tmp1.unfold(0,x_size, shift).transpose(1,2)
            df_X = torch.cat([df_X, tmp1],0)
            
    df_Y = False
    for i in range(len(df)):
        if df_Y is False:
            df_Y = torch.tensor(df[i][y_columns][(x_size-1):].values, dtype=torch.float32)
            df_Y = df_Y.unfold(0, 1, shift).transpose(1,2)
        
        else:
            tmp2 = torch.tensor(df[i][y_columns][(x_size-1):].values, dtype=torch.float32)
            tmp2 = tmp2.unfold(0, 1, shift).transpose(1,2)
            df_Y = torch.cat([df_Y, tmp2],0)
            
    return df_X, df_Y

In [12]:
x_size, shift = 1, 1

In [13]:
train_X, train_Y = window_torch(train, x_size, shift)
train_dataset = torch.utils.data.TensorDataset(train_X, train_Y)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1, shuffle=False) 

test_X, test_Y = window_torch(test, x_size, shift)
test_dataset = torch.utils.data.TensorDataset(test_X, test_Y)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False) 

val_X, val_Y = window_torch(val, x_size, shift)
val_dataset = torch.utils.data.TensorDataset(val_X, val_Y)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

In [33]:
for i, data in enumerate(train_loader):
    print(data)
    print(data[0].dim())
    print(data[0].size())
    print(data[0].shape)
    print(train_X.shape)
    print(train_X[1])
    
   
    
        
    break

[tensor([[[  27.9010,  960.0000,   13.0000,    2.9000,   70.0000,    5.0000,
          2898.0000]]]), tensor([[[1.4800]]])]
3
torch.Size([1, 1, 7])
torch.Size([1, 1, 7])
torch.Size([36836, 1, 7])
tensor([[  27.9010,  960.0000,   13.0000,    2.9000,   70.0000,    5.0000,
         2898.0000]])


In [46]:
train_x = train_X.reshape(36836, 7)
train_y = train_Y.reshape(36836, 1)
print(train_x.shape)
print(train_y.shape)

torch.Size([36836, 7])
torch.Size([36836, 1])


In [23]:
## print(len(list(train_X)))
print(len(list(train_Y)))
print(len(train_X))


36836
36836


In [57]:
model = nn.Linear(7,1)
loss = nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr = 0.0001)


In [58]:
def lrmodel(num_epochs, model, loss_fn, optimizer, train_dl):
    for epoch in range(num_epochs):
        pred = model(train_x)
        loss = loss_fn(pred, train_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ''''for xb,yb in train_dl:
            pred = model(xb)
            #loss = loss_fn(pred, yb)
            loss = F.mse_loss(pred,yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()'''            
        
        print('Epoch [{}/{}], Loss: {:.10f}'.format(epoch+1, num_epochs, loss.item()))
    
    return loss.item()

num_epochs=100
a = lrmodel(num_epochs, model, loss, optim, train_loader)

Epoch [1/100], Loss: 1132802.0000000000
Epoch [2/100], Loss: 46515229818880.0000000000
Epoch [3/100], Loss: 1912548861042497683456.0000000000
Epoch [4/100], Loss: 78631203444798302872435425280.0000000000
Epoch [5/100], Loss: inf
Epoch [6/100], Loss: inf
Epoch [7/100], Loss: inf
Epoch [8/100], Loss: inf
Epoch [9/100], Loss: inf
Epoch [10/100], Loss: inf
Epoch [11/100], Loss: inf
Epoch [12/100], Loss: nan
Epoch [13/100], Loss: nan
Epoch [14/100], Loss: nan
Epoch [15/100], Loss: nan
Epoch [16/100], Loss: nan
Epoch [17/100], Loss: nan
Epoch [18/100], Loss: nan
Epoch [19/100], Loss: nan
Epoch [20/100], Loss: nan
Epoch [21/100], Loss: nan
Epoch [22/100], Loss: nan
Epoch [23/100], Loss: nan
Epoch [24/100], Loss: nan
Epoch [25/100], Loss: nan
Epoch [26/100], Loss: nan
Epoch [27/100], Loss: nan
Epoch [28/100], Loss: nan
Epoch [29/100], Loss: nan
Epoch [30/100], Loss: nan
Epoch [31/100], Loss: nan
Epoch [32/100], Loss: nan
Epoch [33/100], Loss: nan
Epoch [34/100], Loss: nan
Epoch [35/100], Loss:

In [39]:
print(a)

nan
